In [2]:
import os
from Bio import SeqIO
from Bio.SeqUtils import MeltingTemp as mt, gc_fraction
import primer3
import nupack

# Function to read sequences from a FASTA file
def read_fasta(file_path):
    sequences = []
    for record in SeqIO.parse(file_path, "fasta"):
        sequences.append(str(record.seq))
    return sequences  # Returns a list of sequences (if multiple entries)

# Function to generate candidate primers
def generate_primers(sequence, primer_length=30):
    primers = []
    for i in range(len(sequence) - primer_length + 1):
        primer_seq = sequence[i:i + primer_length]
        primers.append(str(primer_seq))
    return primers

# Function to analyze primer secondary structure using NUPACK
def analyze_structure(primer):
    model = nupack.Model(material='dna', celsius=37)
    mfe_struct = nupack.mfe([primer], model=model)
    return mfe_struct[0].energy  # Gibbs Free Energy (∆G)

# Function to evaluate primers
# Function to evaluate primers
def evaluate_primers(primers):
    evaluated = []
    for primer in primers:
        tm = mt.Tm_NN(primer)  # Nearest Neighbor Tm Calculation
        gc = gc_fraction(primer) * 100  # Convert gc_fraction to percentage
        hairpin = primer3.calc_hairpin(primer).tm  # Hairpin Melting Temperature
        homodimer = primer3.calc_homodimer(primer).tm  # Self-dimer Tm
        stability = analyze_structure(primer)  # Gibbs Free Energy (∆G)

        # Debugging Output
        print(f"Primer: {primer}, Tm: {tm:.2f}, GC: {gc:.2f}%, Hairpin: {hairpin:.2f}, Homodimer: {homodimer:.2f}, ΔG: {stability:.2f}")

        # Ensure RPA-compatible thermodynamic properties
        if 37 <= tm <= 42 and 30 <= gc <= 50 and hairpin < 35 and homodimer < 35:
            evaluated.append((primer, tm, gc, hairpin, homodimer, stability))

    return evaluated


def design_rpa_primers(fasta_file):
    fasta_sequences = read_fasta(fasta_file)
    best_primers = []

    for sequence in fasta_sequences:
        candidates = generate_primers(sequence)
        evaluated = evaluate_primers(candidates)

        # Rank by lowest Gibbs Free Energy (most stable)
        sorted_primers = sorted(evaluated, key=lambda x: x[5])
        best_primers.extend(sorted_primers[:5])  # Keep the top 5 per sequence

    return best_primers

# Main execution block
if __name__ == "__main__":
    fasta_file = "genome.fasta"  # Change this to your FASTA file
    if not os.path.exists(fasta_file):
        print(f"Error: {fasta_file} not found!")
    else:
        best_primers = design_rpa_primers(fasta_file)
        for primer in best_primers:
            print(f"Primer: {primer[0]}, Tm: {primer[1]:.2f}, GC: {primer[2]:.2f}%, ΔG: {primer[5]:.2f}")


Primer: ATGAGCAAGGACGCCACCAAAGAAATCTCC, Tm: 62.68, GC: 50.00%, Hairpin: 0.00, Homodimer: -38.45, ΔG: -0.59
Primer: TGAGCAAGGACGCCACCAAAGAAATCTCCG, Tm: 64.24, GC: 53.33%, Hairpin: 0.00, Homodimer: -38.45, ΔG: -0.59
Primer: GAGCAAGGACGCCACCAAAGAAATCTCCGC, Tm: 65.08, GC: 56.67%, Hairpin: 0.00, Homodimer: -38.45, ΔG: -0.59
Primer: AGCAAGGACGCCACCAAAGAAATCTCCGCC, Tm: 66.08, GC: 56.67%, Hairpin: 0.00, Homodimer: -38.45, ΔG: -0.59
Primer: GCAAGGACGCCACCAAAGAAATCTCCGCCC, Tm: 66.68, GC: 60.00%, Hairpin: 0.00, Homodimer: -38.45, ΔG: -0.59
Primer: CAAGGACGCCACCAAAGAAATCTCCGCCCC, Tm: 66.30, GC: 60.00%, Hairpin: 0.00, Homodimer: -38.45, ΔG: -0.59
Primer: AAGGACGCCACCAAAGAAATCTCCGCCCCC, Tm: 67.14, GC: 60.00%, Hairpin: 0.00, Homodimer: -38.45, ΔG: -0.59
Primer: AGGACGCCACCAAAGAAATCTCCGCCCCCA, Tm: 67.97, GC: 60.00%, Hairpin: 0.00, Homodimer: -38.45, ΔG: -0.59
Primer: GGACGCCACCAAAGAAATCTCCGCCCCCAC, Tm: 67.71, GC: 63.33%, Hairpin: 0.00, Homodimer: -51.99, ΔG: 0.00
Primer: GACGCCACCAAAGAAATCTCCGCCCCCACC

KeyboardInterrupt: 

In [ ]:
import os
from Bio import SeqIO
from Bio.SeqUtils import MeltingTemp as mt, gc_fraction
import primer3
import nupack

def read_fasta(file_path):
    sequences = []
    for record in SeqIO.parse(file_path, "fasta"):
        sequences.append(str(record.seq))
    print(f"Loaded {len(sequences)} sequences from {file_path}")
    return sequences

def generate_primers(sequence, primer_length=30):
    primers = []
    for i in range(len(sequence) - primer_length + 1):
        primer_seq = sequence[i:i + primer_length]
        primers.append(str(primer_seq))
    print(f"Generated {len(primers)} candidate primers")
    return primers

def reverse_complement(seq):
    complement = {'A': 'T', 'T': 'A', 'C': 'G', 'G': 'C'}
    rev_comp = "".join(complement[base] for base in reversed(seq))
    print(f"Reverse complement of {seq[:10]}... is {rev_comp[:10]}...")
    return rev_comp


def evaluate_primers(primers):
    evaluated = []
    for primer in primers:
        tm = mt.Tm_NN(primer)
        gc = gc_fraction(primer) * 100
        hairpin = primer3.calc_hairpin(primer).tm
        homodimer = primer3.calc_homodimer(primer).tm
        stability = analyze_structure(primer)

        print(f"Primer: {primer[:10]}..., Tm: {tm:.2f}, GC: {gc:.2f}%, Hairpin: {hairpin:.2f}, Homodimer: {homodimer:.2f}, ΔG: {stability:.2f}")
        
        if 50 <= tm <= 65 and 30 <= gc <= 50 and hairpin < 35 and homodimer < 35:
            evaluated.append((primer, tm, gc, hairpin, homodimer, stability))

    print(f"Selected {len(evaluated)} valid primers")
    return evaluated

def design_rpa_primers(fasta_file):
    fasta_sequences = read_fasta(fasta_file)
    best_primers = []

    for sequence in fasta_sequences:
        print(f"Processing sequence: {sequence[:50]}...")
        candidates = generate_primers(sequence)
        evaluated = evaluate_primers(candidates)
        sorted_primers = sorted(evaluated, key=lambda x: x[5])

        forward_primers = sorted_primers[:5]
        
        reverse_candidates = [reverse_complement(primer[0]) for primer in forward_primers]
        evaluated_reverse = evaluate_primers(reverse_candidates)
        reverse_primers = sorted(evaluated_reverse, key=lambda x: x[5])[:5]

        best_primers.extend(zip(forward_primers, reverse_primers))

    return best_primers

if __name__ == "__main__":
    fasta_file = "genome.fasta"
    if not os.path.exists(fasta_file):
        print(f"Error: {fasta_file} not found!")
    else:
        best_primers = design_rpa_primers(fasta_file)
        for fwd, rev in best_primers:
            print(f"Forward Primer: {fwd[0]}, Tm: {fwd[1]:.2f}, GC: {fwd[2]:.2f}%, ΔG: {fwd[5]:.2f}")
            print(f"Reverse Primer: {rev[0]}, Tm: {rev[1]:.2f}, GC: {rev[2]:.2f}%, ΔG: {rev[5]:.2f}\n")

Loaded 1 sequences from genome.fasta
Processing sequence: ATGAGCAAGGACGCCACCAAAGAAATCTCCGCCCCCACCGACGCCAAGGA...
Generated 1063 candidate primers
Analyzed structure for primer ATGAGCAAGG... ΔG: -0.59
Primer: ATGAGCAAGG..., Tm: 62.68, GC: 50.00%, Hairpin: 0.00, Homodimer: -38.45, ΔG: -0.59
Analyzed structure for primer TGAGCAAGGA... ΔG: -0.59
Primer: TGAGCAAGGA..., Tm: 64.24, GC: 53.33%, Hairpin: 0.00, Homodimer: -38.45, ΔG: -0.59
Analyzed structure for primer GAGCAAGGAC... ΔG: -0.59
Primer: GAGCAAGGAC..., Tm: 65.08, GC: 56.67%, Hairpin: 0.00, Homodimer: -38.45, ΔG: -0.59
Analyzed structure for primer AGCAAGGACG... ΔG: -0.59
Primer: AGCAAGGACG..., Tm: 66.08, GC: 56.67%, Hairpin: 0.00, Homodimer: -38.45, ΔG: -0.59
Analyzed structure for primer GCAAGGACGC... ΔG: -0.59
Primer: GCAAGGACGC..., Tm: 66.68, GC: 60.00%, Hairpin: 0.00, Homodimer: -38.45, ΔG: -0.59
Analyzed structure for primer CAAGGACGCC... ΔG: -0.59
Primer: CAAGGACGCC..., Tm: 66.30, GC: 60.00%, Hairpin: 0.00, Homodimer: -38.45, Δ

In [ ]:
import os
import sys
import argparse
import random
import pandas as pd
from Bio import SeqIO
from Bio.SeqUtils import MeltingTemp as mt, gc_fraction
from Bio.Blast import NCBIWWW, NCBIXML
import primer3
import nupack

# Function to read and process the FASTA file
def read_fasta(file_path):
    sequences = []
    for record in SeqIO.parse(file_path, "fasta"):
        sequences.append(str(record.seq))
    print(f"Loaded {len(sequences)} sequences from {file_path}")
    return sequences

# Function to generate candidate primers
def generate_primers(sequence, primer_length=30):
    primers = [sequence[i:i + primer_length] for i in range(len(sequence) - primer_length + 1)]
    print(f"Generated {len(primers)} candidate primers")
    return primers

# Function to get the reverse complement of a sequence
def reverse_complement(seq):
    complement = {'A': 'T', 'T': 'A', 'C': 'G', 'G': 'C'}
    return "".join(complement[base] for base in reversed(seq))

# Function to analyze primer thermodynamic stability
def analyze_structure(primer):
    model = nupack.Model(material='dna', celsius=37)
    mfe_struct = nupack.mfe([primer], model=model)
    return mfe_struct[0].energy  # Free energy (ΔG)

# Function to evaluate primers based on Tm, GC, secondary structure, and stability
def evaluate_primers(primers):
    evaluated = []
    for primer in primers:
        tm = mt.Tm_NN(primer)
        gc = gc_fraction(primer) * 100
        hairpin = primer3.calc_hairpin(primer).tm
        homodimer = primer3.calc_homodimer(primer).tm
        stability = analyze_structure(primer)

        # Primer selection criteria
        if 30 <= gc <= 50 and 50 <= tm <= 65 and hairpin < 35 and homodimer < 35 and -12 < stability < -6:
            evaluated.append((primer, tm, gc, hairpin, homodimer, stability))

    print(f"Selected {len(evaluated)} valid primers")
    return evaluated

# Function to check background binding using BLAST (NCBI)
def blast_background_check(primer, evalue=0.01):
    """ Runs BLAST against NCBI to check for off-target binding. """
    try:
        print(f"Running BLAST for primer: {primer[:10]}...")
        result_handle = NCBIWWW.qblast("blastn", "nt", primer, expect=evalue)

        # Parse BLAST results
        blast_records = NCBIXML.read(result_handle)
        hits = [alignment for alignment in blast_records.alignments]

        # True if no significant hits (specific primer)
        return len(hits) == 0  
    except Exception as e:
        print(f"BLAST Error: {e}")
        return False  # Assume non-specific if error occurs

# Main function to design primers
def design_rpa_primers(fasta_file):
    fasta_sequences = read_fasta(fasta_file)
    best_primers = []

    for sequence in fasta_sequences:
        print(f"Processing sequence: {sequence[:50]}...")
        candidates = generate_primers(sequence)
        evaluated = evaluate_primers(candidates)
        sorted_primers = sorted(evaluated, key=lambda x: x[5])  # Sort by ΔG (stability)

        # Select top forward primers with BLAST filtering
        forward_primers = []
        for primer in sorted_primers:
            if blast_background_check(primer[0]):  
                forward_primers.append(primer)
            if len(forward_primers) >= 5:
                break

        # Generate reverse primers
        reverse_candidates = [reverse_complement(primer[0]) for primer in forward_primers]
        evaluated_reverse = evaluate_primers(reverse_candidates)
        reverse_primers = sorted(evaluated_reverse, key=lambda x: x[5])[:5]

        best_primers.extend(zip(forward_primers, reverse_primers))

    return best_primers

# Main execution
if __name__ == "__main__":
    parser = argparse.ArgumentParser(description="Design RPA primers with BLAST and secondary structure filtering")
    parser.add_argument("--fasta", required=True, help="Path to input FASTA file")
    args = parser.parse_args()

    if not os.path.exists(args.fasta):
        print(f"Error: {args.fasta} not found!")
        sys.exit(1)

    best_primers = design_rpa_primers(args.fasta)

    # Output final primer pair
    for fwd, rev in best_primers:
        print(f"Forward Primer: {fwd[0]}, Tm: {fwd[1]:.2f}, GC: {fwd[2]:.2f}%, ΔG: {fwd[5]:.2f}")
        print(f"Reverse Primer: {rev[0]}, Tm: {rev[1]:.2f}, GC: {rev[2]:.2f}%, ΔG: {rev[5]:.2f}\n")


Loaded 0 sequences from /Users/harsha/Library/Jupyter/runtime/kernel-v3a8a4dae9f3930a7dbb25f35b5c1c4c6050593639.json
